In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
pd.set_option('display.max_rows', 1000)
%matplotlib inline

In [2]:
files = {
    'FullScan': 'full_scan',
    'ProgressiveIndex': 'progressive_index-wa',
    'ProgressiveIndexWD': 'progressive_index-wd',
    'FullScanCandidate': 'full_scan_cl'
}
def read_df(experiment, name, delta='', partition=''):
    df = 0
    if delta == '':
        df = pd.read_csv(f"results/{name}-{experiment}.csv")
    else:
        df = pd.read_csv(f"results/{name}-{delta}-{partition}-{experiment}.csv")
    repetitions = df['repetition'].max() + 1
    step = int(len(df.index)/repetitions)
    df_final = df[:step].copy().reset_index()
    for rep in range(1, repetitions):
        df_final += df[step * (rep) : step * (rep + 1)].copy().reset_index()
    
    df_final = df_final/repetitions
    
    if 'index_search_time' not in df_final:
        df_final['index_search_time'] = 0.0
    df_final['query_time'] = df_final['initialization_time'] + df_final['index_search_time'] + df_final['scan_time'] + df_final['adaptation_time']
    df_final['query_time_cumsum'] = df_final['query_time'].cumsum()
    return df_final

# First Query Cost
First Query Cost for different deltas

In [3]:
fig = go.Figure()

def plot_time(fig, df, name,deltas):
    fig.add_trace(go.Scatter(x = deltas, y=df[0], mode='lines',name=name))

cols = ['2cols','4cols','8cols']
deltas = ['0.005','0.01','0.05','0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9','1.0']

# for col in cols:
#     fs_r = []
#     fs = read_df(col, files['FullScanCandidate'])
#     for x in range (13):
#         fs_r.append([fs['scan_time'][0]])
#     dfr = pd.DataFrame(data=fs_r)

#     plot_time(fig, dfr, 'fs' +  col.split('c')[0]+' columns',deltas)
for col in cols:
    init_query = []
    for delt in deltas:
        df = read_df(col, files['ProgressiveIndexWD'], delta=delt, partition='1024')
        init_query.append([df['query_time'][0]])
    df = pd.DataFrame(data=init_query)
    plot_time(fig, df, col.split('c')[0]+' columns',deltas)
fig.update_layout(title='First Query',
                   xaxis_title='Delta',
                   yaxis_title='Time (seconds)',xaxis_type="log")
fig.show()

# Total Query Cost
Total Query Cost for different deltas

In [4]:
fig = go.Figure()

def plot_time(fig, df, name,deltas):
    fig.add_trace(go.Scatter(x = deltas, y=df[0], mode='lines',name=name))

cols = ['2cols','4cols','8cols']
deltas = ['0.005','0.01','0.05','0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9','1.0']

for col in cols:
    init_query = []
    for delt in deltas:
        df = read_df(col, files['ProgressiveIndexWD'], delta=delt, partition='1024')
        init_query.append([df['scan_time'].sum()])
    df = pd.DataFrame(data=init_query)
    plot_time(fig, df, col.split('c')[0]+' columns',deltas)
fig.update_layout(title='Total Cost ',
                   xaxis_title='Delta',
                   yaxis_title='Time (seconds)',xaxis_type="log")
fig.show()

# Convergence (#Queries)
Number of queries to achieve convergence

In [5]:
fig = go.Figure()

def plot_time(fig, df, name,deltas):
    fig.add_trace(go.Scatter(x = deltas, y=df[0], mode='lines',name=name))

cols = ['2cols','4cols','8cols']
deltas = ['0.01','0.05','0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1.0']

for col in cols:
    init_query = []
    for delt in deltas:
        df = read_df(col, files['ProgressiveIndexWD'], delta=delt, partition='1024')
        conv = df.index[df['adaptation_time'] == 0].tolist()
        if not conv:
            init_query.append(None)
        else:
            init_query.append(conv[0])
    df = pd.DataFrame(data=init_query)
    plot_time(fig, df, col.split('c')[0]+' columns',deltas)
fig.update_layout(title=' #Query to Convergence',
                   xaxis_title='Delta',
                   yaxis_title='#Queries',xaxis_type="log")
fig.show()

# Pay-Off (#Queries)
Number of queries for indexing to pay off

In [6]:
fig = go.Figure()

def plot_time(fig, df, name,deltas):
    a = 0
    if df.empty:
        a = None
    else:
        a = df[0]
    fig.add_trace(go.Scatter(x = deltas, y=a, mode='lines',name=name))

cols = ['2cols','4cols','8cols']
deltas = ['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1.0']

# for col in cols:
init_query = []
for col in cols:
    fs = read_df(col, files['FullScanCandidate'])
    init_query = []
    for delt in deltas:
        df = read_df(col, files['ProgressiveIndexWD'], delta=delt, partition='1024')
        a = np.where(df['query_time_cumsum'].to_numpy()*1.05 < fs['query_time_cumsum'].to_numpy() )
        init_query.append(a[0])
    df = pd.DataFrame(data=init_query)
    plot_time(fig, df, col.split('c')[0]+' columns',deltas)
    
fig.update_layout(title=' #Query to payoff',
                   xaxis_title='Delta',
                   yaxis_title='#Queries')
fig.show()